## Import libraries

In [7]:
import numpy as np
import pandas as pd
import re
import numpy as np
import pandas as pd
import numpy as np
import string
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

In [3]:
train = pd.read_csv("train.csv")
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
cols_target = ['obscene','insult','toxic','severe_toxic','identity_hate','threat']

## Cleaning

In [8]:
def clean_text(text):
    
    text = text.lower()
    
    #pattern = [zero or more character]
    text = re.sub('\[.*?\]', '', text)
    
    #pattern = with or without(http),://, one or more non-white space character, OR www, .,one or more non-white space character
    text = re.sub('https?://\S+|www\.\S+', '', text)
    
    #pattern = <, zero or more characters, >, (one or more occurance of >)
    text = re.sub('<.*?>+', '', text)
    
    #pattern = any punctionation
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    
    #pattern = any new line
    text = re.sub('\n', '', text)
    
    #pattern = any from[a-zA-Z0-9_], any from[0-9], any from [a-zA-Z0-9_]
    text = re.sub('\w*\d\w*', '', text)
    return text

In [9]:
train['clean_text'] = train['comment_text'].apply(str).apply(lambda x: clean_text(x))

In [10]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,explanationwhy the edits made under my usernam...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,daww he matches this background colour im seem...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,hey man im really not trying to edit war its j...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,morei cant make any real suggestions on improv...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,you sir are my hero any chance you remember wh...


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, stop_words='english')
X_vec_train = vectorizer.fit_transform(train['clean_text'])
X_vec_test = vectorizer.transform(test['clean_text'])

In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
logreg = LogisticRegression(C=12.0, solver='liblinear')


## Tokenization

In [11]:
tokenization = nltk.tokenize.RegexpTokenizer(r'\w+')
train['clean_text'] = train['clean_text'].apply(lambda x: tokenization.tokenize(x))
train['clean_text']

0         [explanationwhy, the, edits, made, under, my, ...
1         [daww, he, matches, this, background, colour, ...
2         [hey, man, im, really, not, trying, to, edit, ...
3         [morei, cant, make, any, real, suggestions, on...
4         [you, sir, are, my, hero, any, chance, you, re...
                                ...                        
159566    [and, for, the, second, time, of, asking, when...
159567    [you, should, be, ashamed, of, yourself, that,...
159568    [spitzer, umm, theres, no, actual, article, fo...
159569    [and, it, looks, like, it, was, actually, you,...
159570    [and, i, really, dont, think, you, understand,...
Name: clean_text, Length: 159571, dtype: object

## Stop words

In [12]:
stop_words = set(stopwords.words('english'))
def removing_stopwords(token):
    return [w for w in token if not w.lower() in stop_words]

In [13]:
train['clean_text'] = train['clean_text'].apply(lambda x : removing_stopwords(x))
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanationwhy, edits, made, username, hardco..."
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[daww, matches, background, colour, im, seemin..."
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, im, really, trying, edit, war, guy,..."
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[morei, cant, make, real, suggestions, improve..."
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page, thats]"


## Lematization

In [14]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatize_word(tokenization):
    return [wordnet_lemmatizer.lemmatize(token, pos="v") for token in tokenization]

In [15]:
train['Text_Lemmatization'] = train['clean_text'].apply(lambda x : lemmatize_word(x))
train['Text_Lemmatization']

0         [explanationwhy, edit, make, username, hardcor...
1         [daww, match, background, colour, im, seemingl...
2         [hey, man, im, really, try, edit, war, guy, co...
3         [morei, cant, make, real, suggestions, improve...
4                [sir, hero, chance, remember, page, thats]
                                ...                        
159566    [second, time, ask, view, completely, contradi...
159567          [ashamed, horrible, thing, put, talk, page]
159568    [spitzer, umm, theres, actual, article, prosti...
159569    [look, like, actually, put, speedy, first, ver...
159570    [really, dont, think, understand, come, idea, ...
Name: Text_Lemmatization, Length: 159571, dtype: object

In [16]:
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

train['Final_text'] = train['Text_Lemmatization'].apply(lambda x : combine_text(x))
train['Final_text']

0         explanationwhy edit make username hardcore met...
1         daww match background colour im seemingly stic...
2         hey man im really try edit war guy constantly ...
3         morei cant make real suggestions improvement w...
4                       sir hero chance remember page thats
                                ...                        
159566    second time ask view completely contradict cov...
159567                 ashamed horrible thing put talk page
159568    spitzer umm theres actual article prostitution...
159569    look like actually put speedy first version de...
159570    really dont think understand come idea bad rig...
Name: Final_text, Length: 159571, dtype: object

In [17]:
def classify(row):
    for c in categories:
        if row[c]==1:
            return 1
    return 0

In [18]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,Text_Lemmatization,Final_text
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanationwhy, edits, made, username, hardco...","[explanationwhy, edit, make, username, hardcor...",explanationwhy edit make username hardcore met...
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[daww, matches, background, colour, im, seemin...","[daww, match, background, colour, im, seemingl...",daww match background colour im seemingly stic...
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, im, really, trying, edit, war, guy,...","[hey, man, im, really, try, edit, war, guy, co...",hey man im really try edit war guy constantly ...
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[morei, cant, make, real, suggestions, improve...","[morei, cant, make, real, suggestions, improve...",morei cant make real suggestions improvement w...
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page, thats]","[sir, hero, chance, remember, page, thats]",sir hero chance remember page thats
...,...,...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,"[second, time, asking, view, completely, contr...","[second, time, ask, view, completely, contradi...",second time ask view completely contradict cov...
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,"[ashamed, horrible, thing, put, talk, page]","[ashamed, horrible, thing, put, talk, page]",ashamed horrible thing put talk page
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,"[spitzer, umm, theres, actual, article, prosti...","[spitzer, umm, theres, actual, article, prosti...",spitzer umm theres actual article prostitution...
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,"[looks, like, actually, put, speedy, first, ve...","[look, like, actually, put, speedy, first, ver...",look like actually put speedy first version de...


In [19]:
categories = [x for x in train.columns]
categories

['id',
 'comment_text',
 'toxic',
 'severe_toxic',
 'obscene',
 'threat',
 'insult',
 'identity_hate',
 'clean_text',
 'Text_Lemmatization',
 'Final_text']

In [20]:
train['Toxic_or_not'] = train[categories].apply(classify,axis=1)

In [21]:
train

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,clean_text,Text_Lemmatization,Final_text,Toxic_or_not
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,"[explanationwhy, edits, made, username, hardco...","[explanationwhy, edit, make, username, hardcor...",explanationwhy edit make username hardcore met...,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,"[daww, matches, background, colour, im, seemin...","[daww, match, background, colour, im, seemingl...",daww match background colour im seemingly stic...,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,"[hey, man, im, really, trying, edit, war, guy,...","[hey, man, im, really, try, edit, war, guy, co...",hey man im really try edit war guy constantly ...,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,"[morei, cant, make, real, suggestions, improve...","[morei, cant, make, real, suggestions, improve...",morei cant make real suggestions improvement w...,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,"[sir, hero, chance, remember, page, thats]","[sir, hero, chance, remember, page, thats]",sir hero chance remember page thats,0
...,...,...,...,...,...,...,...,...,...,...,...,...
159566,ffe987279560d7ff,""":::::And for the second time of asking, when ...",0,0,0,0,0,0,"[second, time, asking, view, completely, contr...","[second, time, ask, view, completely, contradi...",second time ask view completely contradict cov...,0
159567,ffea4adeee384e90,You should be ashamed of yourself \n\nThat is ...,0,0,0,0,0,0,"[ashamed, horrible, thing, put, talk, page]","[ashamed, horrible, thing, put, talk, page]",ashamed horrible thing put talk page,0
159568,ffee36eab5c267c9,"Spitzer \n\nUmm, theres no actual article for ...",0,0,0,0,0,0,"[spitzer, umm, theres, actual, article, prosti...","[spitzer, umm, theres, actual, article, prosti...",spitzer umm theres actual article prostitution...,0
159569,fff125370e4aaaf3,And it looks like it was actually you who put ...,0,0,0,0,0,0,"[looks, like, actually, put, speedy, first, ve...","[look, like, actually, put, speedy, first, ver...",look like actually put speedy first version de...,0


In [22]:
from sklearn.pipeline import Pipeline

In [29]:
train.shape

(159571, 12)

In [24]:
x_train,x_test,y_train,y_test = train_test_split(train['Final_text'],train['Toxic_or_not'],test_size=0.2,random_state=42)

In [25]:
pipe_lr = Pipeline(steps=[('cv',TfidfVectorizer()),('lr',LogisticRegression())])

In [26]:
pipe_lr.fit(x_train,y_train)

c:\users\пользователь\pycharmprojects\selenium_project\venv\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('cv', TfidfVectorizer()), ('lr', LogisticRegression())])

In [27]:
pipe_lr.score(x_test,y_test)

0.9547234842550525

## Example

In [44]:
ex1 = "You are cool"
ex2 = "You are stupid"
ex3 = "You are fool"

In [31]:
pipe_lr.predict([ex1])

array([0], dtype=int64)

In [32]:
pipe_lr.predict([ex2])

array([1], dtype=int64)

In [45]:
pipe_lr.predict([ex3])

array([1], dtype=int64)

In [46]:
pipe_lr.predict([ex3])[0]

1

In [47]:
import joblib
pipeline_file = open("toxic_classifier_model.pkl","wb")
joblib.dump(pipe_lr,pipeline_file)
pipeline_file.close()